## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [5]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

c:\Users\jeongseunghyun\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-NQ-W-e1F-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [6]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [7]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [ ]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
# prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")
prompt = PromptTemplate.from_template("{topic} 에 대해 재미있게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

In [ ]:
chain

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
# input = {"topic": "인공지능 모델의 학습 원리", "how": "전문가스럽게"}
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="물론이죠! 인공지능 모델의 학습 원리를 재미있게 설명해볼게요.\n\n생각해보세요, 인공지능은 마치 어린 아이가 새로운 것을 배우는 것과 비슷해요. 예를 들어, 그림을 보고 강아지와 고양이를 구별하는 법을 배우는 과정이라고 할 수 있죠.\n\n1. **데이터라는 교과서**: 먼저, 인공지능에게 많은 그림(데이터)을 보여줍니다. 강아지 그림과 고양이 그림이 섞여 있죠. 이 그림들이 바로 인공지능이 배우는 '교과서'입니다.\n\n2. **예측과 정답**: 인공지능은 처음에는 아무것도 몰라서 무작위로 '이건 강아지다' 또는 '이건 고양이다'라고 말할 수 있어요. 그런데 정답(실제 그림이 강아지인지 고양이인지)을 알려주면, 인공지능은 자신의 예측이 맞았는지 확인하죠.\n\n3. **실수에서 배우기**: 만약 틀렸다면, 인공지능은 '아, 내가 틀렸구나!'라고 깨닫고, 틀린 부분을 수정하려고 해요. 이때 '오차'라는 것을 계산하는데, 이 오차를 줄이기 위해서 내부의 '학습 규칙'을 사용해서 가중치라는 값을 조금씩 조정합니다.\n\n4. **반복 학습**: 이 과정을 수천, 수만 번 반복하면서 인공지능은 점점 더 정확하게 그림을 구별할 수 있게 돼요. 마치 어린아이가 여러 번 연습해서 그림을 더 잘 그리게 되는 것과 비슷하죠.\n\n5. **최종 마스터**: 결국, 수많은 반복과 수정 끝에 인공지능은 새로운 그림을 보여줬을 때도 '이건 강아지다' 또는 '이건 고양이다'라고 자신 있게 말할 수 있게 되는 거예요.\n\n요약하자면, 인공지능의 학습은 '많은 예제와 실수, 그리고 반복을 통해 점점 더 똑똑해지는 과정'입니다. 마치 퍼즐을 맞추듯이, 여러 조각들을 맞춰가며 최종 그림을 완성하는 것과 비슷하죠!\n\n혹시 더 궁금한 점 있나요?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 522, 'prompt_tokens':

In [12]:
### 변수가 하나일 경우에는 예외적으로 딕셔너리 형태가 아니여도 가능
chain.invoke('나라별 안녕하세요를 설명해주세요')

AIMessage(content='물론이죠! "안녕하세요"는 한국에서 가장 흔하게 쓰이는 인사말로, 전 세계 어디를 가든지 "Hello" 또는 "Hi"와 비슷한 역할을 합니다. 그런데 나라별로 "안녕하세요"가 어떻게 다르게 표현되고, 어떤 재미있는 이야기가 숨어 있는지 한번 살펴볼까요?\n\n1. 한국: "안녕하세요"  \n한국에서는 정중하고 따뜻한 인사말로, 처음 만난 사람에게도 예의를 갖춰서 씁니다. "안녕"은 평화롭고 안전하다는 의미를 담고 있어요. 그래서 한국 사람들은 인사할 때 "안녕하세요?"라고 묻는 것 자체가 상대방의 평안과 건강을 바라는 마음이 담겨 있답니다.\n\n2. 일본: "こんにちは" (Konnichiwa)  \n일본에서는 낮 인사로 "こんにちは"를 사용하는데, 이 말은 "좋은 하루"라는 의미를 내포하고 있어요. 일본인들은 인사에 담긴 세심한 배려와 예의를 매우 중요시하죠.\n\n3. 중국: "你好" (Nǐ hǎo)  \n중국에서는 "你好"라고 하며, 문자 그대로 "너 좋다"라는 의미입니다. 이 간단한 인사말이 중국인들의 친근함과 정중함을 동시에 보여줍니다.\n\n4. 미국/영어권: "Hello" 또는 "Hi"  \n영어권에서는 "Hello"가 공식적이고, "Hi"는 좀 더 친근하고 캐주얼한 느낌이죠. 재미있는 점은, "Hello"라는 말이 19세기 초에 전화가 발명되면서 널리 퍼졌다는 사실! 전화기에서 처음으로 "Hello"를 사용한 사람이 바로 토마스 에디슨이라는 이야기도 있어요.\n\n5. 프랑스: "Bonjour"  \n프랑스에서는 낮에는 "Bonjour"로 인사하는데, 이 말은 "좋은 날"이라는 의미입니다. 저녁에는 "Bonsoir"로 바꾸죠. 프랑스인들은 인사할 때도 우아함과 예의를 중요시하는 문화가 느껴집니다.\n\n6. 인도: "नमस्ते" (Namaste)  \n인도에서는 손바닥을 모으고 인사하는 "Namaste"를 사용합니다. 이 말은 "내 마음이 당신과 만나다"라는 의미를 담고 있어요. 영적이고 평화로운 느낌이 가득하

아래는 스트리밍을 출력하는 예시 입니다.

In [13]:
# 스트리밍 출력을 위한 요청
# answer = chain.stream(input)
answer = chain.stream('나라별 안녕하세요를 설명해주세요 한글로 어떻게 발음하는지도 함께요')
# 스트리밍 출력
stream_response(answer)

안녕하세요! 나라별로 "안녕하세요"를 어떻게 인사하는지, 그리고 한글로 어떻게 발음하는지 재미있게 살펴볼게요. 준비됐나요? 출발~!

1. **한국 (South Korea)**  
   - 인사말: **안녕하세요**  
   - 발음: [an-nyeong-ha-se-yo]  
   - 설명: 한국에서는 "안녕하세요"라고 말하면 정중하고 친근한 인사예요. 마치 "안녕"에 "하세요"를 붙인 느낌이죠. 마치 친구에게 "안녕?" 하고 묻는 것처럼 부드럽고 따뜻하게 들려요.

2. **중국 (China)**  
   - 인사말: **你好** (니하오)  
   - 발음: [ni-hao]  
   - 설명: 중국에서는 "니하오"라고 해요. "니"는 "너"처럼 들리고, "하오"는 "좋다"라는 의미도 있지만 인사로 쓰여요. 중국어는 한글로 적으면 "니하오"라고 쓰고, 발음도 비슷하게 들리죠!

3. **일본 (Japan)**  
   - 인사말: **こんにちは** (콘니치와)  
   - 발음: [konnichiwa]  
   - 설명: 일본에서는 "콘니치와"라고 해요. "안녕하세요"와 비슷한 의미로 낮 시간에 쓰는 인사죠. 한글로 적으면 "콘니치와"이고, 발음도 비슷하게 들리니 일본 친구에게 인사할 때 딱 좋아요!

4. **미국 (USA)**  
   - 인사말: **Hello**  
   - 발음: [heh-loh]  
   - 설명: 미국에서는 "헬로"라고 해요. 영어로는 "Hello"라고 쓰고, 발음은 "헤로"처럼 들리기도 해요. 친근하고 간단한 인사죠!

5. **스페인 (Spain)**  
   - 인사말: **Hola**  
   - 발음: [o-la]  
   - 설명: 스페인에서는 "올라"라고 해요. "안녕하세요"보다 조금 더 캐주얼하고 친근한 느낌이죠. 한글로 적으면 "올라"이고, 발음도 쉽죠!

6. **프랑스 (France)**  
   - 인사말: **Bonjour**  
   - 발음: [bawn-zhoor]  
   - 설명: 프랑스에서는 "봉쥬

### 출력파서(Output Parser)


In [16]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [ ]:
template_ex = """
{{question}} escape 처리
"""


In [17]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
대답의 인물이 다를 경우 A: ,B: 처럼 표기해주세요
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [18]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

In [19]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream("저는 식당에 가서 음식을 주문하고 싶어요")
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
A: Hello! I’d like to order a meal, please.  
B: Sure! What would you like to have?  
A: I’ll have the grilled chicken with rice, and a glass of water, please.  
B: Certainly. Anything else?  
A: No, that’s all. Thank you.  

- 한글 해석:  
A: 안녕하세요! 저는 음식을 주문하고 싶어요.  
B: 네! 무엇을 드시겠어요?  
A: 구운 치킨과 밥, 그리고 물 한 잔 주세요.  
B: 알겠습니다. 더 필요하신 건요?  
A: 아니요, 이게 전부예요. 감사합니다.

In [23]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 한국 정치에 대한 토론"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: I’ve been reading about South Korea’s recent political developments. What’s your take on the current situation?
B: Well, it’s quite complex. There are many opinions about the government’s policies and their impact on the people.
A: Do you think the protests are justified?
B: In some cases, yes. People have the right to express their dissatisfaction, especially if they feel their voices aren’t being heard.
A: How do you think international observers view these protests?
B: Most see them as a sign of a healthy democracy, but there are concerns about stability and external influences.
A: It’s interesting to see how politics in Korea influence not only their country but also the region.
B: Absolutely. South Korea’s political decisions often have broader implications, especially given its economic power.

- 한글 해석:
A: 저는 최근 한국의 정치적 상황에 관해 읽어봤어요. 현재 상황에 대해 어떻게 생각하세요?
B: 글쎄요, 꽤 복잡하죠. 정부의 정책과 그 정책이 국민에게 미치는 영향에 관한 여러 의견이 있어요.
A: 시위가 정당하다고 보세요?
B: 경우에 따라서는 그렇죠. 사람들이 자신의 불만을 표현할 권리가 있